In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy
import glob
from datetime import datetime


date="20200227"

In [ ]:
## Read
rtt_header = ['timestamp', 'src', 'dst', 'rtt', 'country_code', 'asn', 'server', 'ipv']
rtt_dtypes = {1: str, 2: str, 3: float, 4: str, 5: str, 6:str, 7:str}
rtt_file = f'Data/enriched_rtt_{date}.csv'

df = pd.read_csv(rtt_file, sep=';',
                        names= rtt_header,
                        dtype=rtt_dtypes,
                        parse_dates=[0]
                    )

#df['timestamp'] = pandas.to_datetime(df['timestamp'])
#df = df.drop(columns=['timestamp'])
#print(df.head())

#df.dtypesprint(df_merge.head())
df = df.replace({'dst': {'194.0.12.1': 'A', '2001:678:f::1': 'A'}})
df = df.replace({'dst': {'194.0.13.1': 'B', '2001:678:10::1': 'B'}})
df = df.replace({'dst': {'194.0.14.1': 'C', '2001:678:11::1': 'C'}})
df = df.replace({'dst': {'193.29.206.1': 'D', '2001:678:1::1': 'D'}})
df = df.rename(columns={'dst': 'anycast'})
df.head()

In [ ]:
## Procces rtt file
df = df.groupby(['src', 'anycast', 'country_code','asn','server','ipv']).agg(
                {"rtt":['mean', 'count']}).reset_index()

#Merge columns
df.columns = ["_".join(x) if x[1] != '' else x[0] for x in df.columns.ravel() ]
#df = df.replace({'ipv': {'ipv4': '4', 'ipv6': '6'}})
df.head()


In [ ]:
## Read and procces queries
queries_file = f'Data/queries_{date}/enriched_{date}_cz.csv'
queries_data_type = {0: str, 1: str, 2: str, 3: str, 4: int, 5: int, 6: int, 7: str, 8: str}

df_queries = pd.read_csv(queries_file, sep=';',
                        dtype=queries_data_type,
                    )

#print(df.dtypes)
df_queries = df_queries.replace({'dst': {'194.0.12.1': 'A', '2001:678:f::1': 'A'}})
df_queries = df_queries.replace({'dst': {'194.0.13.1': 'B', '2001:678:10::1': 'B'}})
df_queries = df_queries.replace({'dst': {'194.0.14.1': 'C', '2001:678:11::1': 'C'}})
df_queries = df_queries.replace({'dst': {'193.29.206.1': 'D', '2001:678:1::1': 'D'}})
df_queries = df_queries.rename(columns={'dst': 'anycast'})
df_queries = df_queries.set_index(['src','anycast','server', 'ipv', 'asn', 'country_code'])
#df_queries = df_queries.drop(columns=['ipv'])
df_queries.head()


In [ ]:
## Merge dataframes

df = df.set_index(['src','anycast', 'server', 'ipv', 'asn', 'country_code'])

df_merge = df_queries.join(df, on=['src','anycast', 'server', 'ipv', 'asn', 'country_code'], how='outer')
#print(df_merge.head())

In [ ]:
## Enrich datatable by country code with Country Name, Region, Subregion
cc_file = 'Data/cc.csv'
df_cc = pd.read_csv(cc_file, sep=','
                     )
#print(df_cc.head())
df_cc_short = df_cc[['alpha-2','region','sub-region','name']]
df_cc_short = df_cc_short.rename(columns={"alpha-2": "country_code", "name": "country_name", "sub-region":"sub_region"})
#print(df_cc_short.head())
main_index = ['server', 'ipv', 'asn', 'country_code']
result = pd.merge(df_cc_short.reset_index(), 
                  df_merge.reset_index(),
                  on=['country_code'], 
                  how='outer').set_index(main_index)
#result = result.drop(columns=['src','index'])
result = result.drop(columns=['index']).reset_index()



In [ ]:
def eval_rtt_by_column(x):
    d = {}
    d['rtt_mean'] = x['rtt_mean'].mean()
    d['cnt'] = x['cnt'].sum()
    if x['cnt'].sum() != 0: 
        d['rtt_eval'] = (x['cnt'] * x['rtt_mean']).sum() / x['cnt'].sum()
    else:
        d['rtt_eval'] = None
    return pd.Series(d, index=['rtt_mean', 'rtt_eval', 'cnt', ])

In [ ]:
result = result.groupby(['server', 'anycast', 'ipv', 'asn', 'region', 'sub_region', 'country_name','country_code']).apply(eval_rtt_by_column)

In [ ]:
## Export whole dataframe to csv
result.to_csv(f'dataframes/{date}_src.csv')